In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt import MultiPromptChain

In [2]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=500)

In [3]:
# 2- Templates for each expert
medicine_template = """You are a highly skilled doctor. 
You answer medical questions clearly and accurately. Here is the question: {input}"""

psychology_template = """You are an expert psychologist. 
You answer psychological questions with care and clarity. Here is the question: {input}"""

math_template = """You are an expert mathematician. 
You solve math problems step by step. Here is the question: {input}"""

history_tunisia_template = """You are an expert in the history of Tunisia. 
You answer questions about historical events in Tunisia precisely. Here is the question: {input}"""

prompt_infos = [
    {"name": "medicine", "description": "Medical questions", "prompt_template": medicine_template},
    {"name": "psychology", "description": "Psychological questions", "prompt_template": psychology_template},
    {"name": "math", "description": "Math questions", "prompt_template": math_template},
    {"name": "history_tunisia", "description": "History of Tunisia", "prompt_template": history_tunisia_template}
]

In [4]:
# 3- Create an LLMChain for each expert
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt = ChatPromptTemplate.from_template(template=p_info["prompt_template"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

In [12]:
# 4- Create the router
destinations_str = "\n".join([f"{p['name']}: {p['description']}" for p in prompt_infos])

MULTI_PROMPT_ROUTER_TEMPLATE = f"""
Given a question, select the most appropriate expert for answering.
Available experts:
{destinations_str}

Input: {{input}}

Output a JSON in a markdown code block like this:
```json
{{{{"destination": "string", "next_inputs": "string"}}}}

"""

router_prompt = PromptTemplate(
    template=MULTI_PROMPT_ROUTER_TEMPLATE,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [13]:
# 5- MultiPrompt chain
default_chain = LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template("{input}"))

multi_prompt_chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [15]:
# 6- Test with some questions
questions = [
    "Quels sont les symptômes courants de la grippe ?",
    "Résoudre l'équation x^2 - 5x + 6 = 0",
    "Quelles sont les causes de stress les plus fréquentes selon la psychologie moderne ?",
    "Quand a eu lieu l'indépendance de la Tunisie ?",
    "Calculate 7*8+65"
]

for q in questions:
    print(f"\nQuestion: {q}")
    response = multi_prompt_chain.run(q)
    print(f"Answer: {response}")


Question: Quels sont les symptômes courants de la grippe ?


> Entering new MultiPromptChain chain...
medicine: {'input': 'Quels sont les symptômes courants de la grippe ?'}
> Finished chain.
Answer: Les symptômes courants de la grippe comprennent la fièvre, les frissons, les maux de tête, les courbatures, la fatigue, la toux, les maux de gorge et parfois des nausées et des vomissements. Il est important de consulter un médecin si vous présentez ces symptômes pour obtenir un diagnostic précis et un traitement approprié.

Question: Résoudre l'équation x^2 - 5x + 6 = 0


> Entering new MultiPromptChain chain...
math: {'input': 'x^2 - 5x + 6 = 0'}
> Finished chain.
Answer: To solve the quadratic equation x^2 - 5x + 6 = 0, we can use the quadratic formula:

x = (-b ± √(b^2 - 4ac)) / 2a

In this case, a = 1, b = -5, and c = 6. Plugging these values into the formula, we get:

x = (5 ± √((-5)^2 - 4*1*6)) / 2*1
x = (5 ± √(25 - 24)) / 2
x = (5 ± √1) / 2
x = (5 ± 1) / 2

So, the solutions to th